# Four point functions in the Potts and $O(n)$ models

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [1]:
Pkg.activate("..") # activate the parent environment
using DoubleFloats, BootstrapVirasoro
n_threads = Threads.nthreads()
if n_threads == 1
    println("You are using a single thread. Consider starting julia with more threads, for instance by setting
    the environment variable `export JULIA_NUM_THREADS=auto`")
end

  Activating project at `~/Documents/Recherche/projet_these/code/BootstrapVirasoro`


## Spectra

We create functions to generate the spectra of the $O(n)$ and Potts CFTs, with a cutoff:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^{PSU(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^\text{Potts} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}+2\\ s\in\frac{1}{r}\mathbb{Z}}} \bigcup  \left\{ V_{P_{(0,s)}}\right\}_{s\in \mathbb{N}+\frac12}\ .
\end{align}


In [2]:
setprecision(BigFloat, 50, base=10)
c = CC(β=1/(big"0.8" + big"0.1"*im))
Δmax = 30.

LoopFields = Dict(
    :On => vcat(
        [Field(c, r=r, s=s) for r in 1//2:1//2:50 for s in -10+1//r:1//r:10 if r*s%1 == 0],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:2:50]
    ),
    :PSUn => vcat(
        [Field(c, r=r, s=s) for r in 1:50 for s in -10+1//r:1//r:10],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:50]
    ),
    :Potts => vcat(
        [Field(c, r=r, s=s) for r in 2:50 for s in -10+1//r:1//r:10],
        [Field(c, r=0, s=s, diagonal=true) for s in 1//2:1:3//2],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:50]
    )
)

LoopSpectra = Dict(
    model => Spectrum(LoopFields[model], Δmax, interchiral=true)
    for model in (:On, :PSUn, :Potts)
);

## Bootstrap equations

We numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.

## $\langle (\frac{1}{2}, 0)^2 (1, 0)^2 \rangle$

In [ ]:
indices = ((1//2, 0), (1//2, 0), (1, 0), (1, 0))
fields = [Field(c, r=r, s=s) for (r, s) in indices]
@time co = Correlation(fields..., Δmax=Δmax)

Schan_On = ChannelSpectra(
    co, LoopSpectra[:On],
    signature = Dict(:s => 1, :t => 1//2, :u => 3//2),
)
@time b = BootstrapSystem(Schan_On);
compute_linear_system!(b)
@time solve!(b, precision_factor=1)
b.consts

158.487810 seconds (2.21 G allocations: 98.324 GiB, 9.57% gc time, 5.21% compilation time)


┌ Info: system size: (118, 112)
└ @ BootstrapVirasoro /Users/paulroux/Documents/Recherche/projet_these/code/BootstrapVirasoro/src/bootstrap_equations.jl:139


  9.213506 seconds (53.37 M allocations: 2.375 GiB, 4.59% gc time, 65.81% compilation time)


Channel s
Fields         | Structure constants          | Errors                     
--------------------------------------------------------------------
V_{(1, 0)}     |  1.00000e+00 + 0.00000e+00im | 0.00000e+00 + 0.00000e+00im
V_{(1, 1)}     |  5.02183e-01 - 8.30317e-02im |  1.29577e-19 + 0.00000e+00im
V_{(2, -1//2)} |  2.73704e-03 + 2.00265e-02im |  9.36877e-17 + 0.00000e+00im
V_{(2, 0)}     |  2.39155e-03 + 2.55139e-02im |  1.10100e-16 + 0.00000e+00im
V_{(2, 1//2)}  |  2.73704e-03 + 2.00265e-02im |  1.06242e-16 + 0.00000e+00im
V_{(2, 1)}     | -3.97586e-19 + 1.25531e-19im |  1.31370e+00 + 0.00000e+00im
V_{(3, -2//3)} | -6.00257e-07 + 2.35486e-07im |  1.69845e-09 + 0.00000e+00im
V_{(3, -1//3)} | -7.07647e-07 - 9.43683e-07im |  1.27692e-09 + 0.00000e+00im
V_{(3, 0)}     | -6.40312e-07 - 1.92164e-06im |  7.85826e-10 + 0.00000e+00im
V_{(3, 1//3)}  | -7.07647e-07 - 9.43683e-07im |  1.13800e-09 + 0.00000e+00im
V_{(3, 2//3)}  | -6.00257e-07 + 2.35486e-07im |  1.51100e-09 + 0.00000e+00im

In [ ]:
evaluate(b.blocks[:s][1], b.positions[1])

MethodError: MethodError: no method matching evaluate(::Field{Complex{BigFloat}}, ::Complex{BigFloat})
The function `evaluate` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  evaluate(!Matched::BootstrapVirasoro.BlockInterchiral, ::T) where T
   @ BootstrapVirasoro ~/Documents/Recherche/projet_these/code/BootstrapVirasoro/src/ConformalBlocks/evaluate.jl:95
  evaluate(!Matched::BootstrapVirasoro.BlockLogarithmic, ::T) where T
   @ BootstrapVirasoro ~/Documents/Recherche/projet_these/code/BootstrapVirasoro/src/ConformalBlocks/evaluate.jl:64
  evaluate(!Matched::BootstrapVirasoro.BlockFactorized, ::T) where T
   @ BootstrapVirasoro ~/Documents/Recherche/projet_these/code/BootstrapVirasoro/src/ConformalBlocks/evaluate.jl:60
  ...


In [12]:
b.blocks[:s]

Dict{Field{Complex{BigFloat}}, Vector{Complex{BigFloat}}} with 42 entries:
  V_{(4, -1//2)} => [-1.46587e-05-0.000223579im, -0.000247924-9.86149e-05im, 3.…
  V_{(6, -2//3)} => [-4.48566e-09-3.05187e-09im, 1.7116e-09+9.9696e-09im, 1.582…
  V_{(6, 1//2)}  => [2.60275e-09+4.18758e-09im, -7.53573e-09+4.78335e-09im, 3.5…
  V_{(2, -1//2)} => [-0.0284754+0.127025im, 0.0558019+0.15517im, -0.0048485+0.0…
  V_{(3, 0)}     => [0.00309697+0.00947491im, 0.00420398+0.011321im, -7.4643e-0…
  V_{(4, 1//2)}  => [0.00022619-7.99777e-05im, 0.000192819+0.000215779im, -3.96…
  V_{(5, 4//5)}  => [-1.01188e-07-9.29609e-07im, 1.62697e-06+3.68528e-07im, -1.…
  V_{(6, -1//3)} => [1.88742e-09+4.63139e-09im, 9.57319e-09-8.05171e-10im, 6.95…
  V_{(6, -5//6)} => [2.11521e-09-5.76504e-09im, -6.7629e-09+8.65756e-09im, 8.03…
  V_{(6, 5//6)}  => [-5.59366e-10-5.91617e-09im, 6.29273e-09+7.82276e-09im, 1.6…
  V_{(2, 1)}     => [0.0636676-0.113701im, 0.12481-0.224915im, 0.0225054-0.0138…
  V_{(5, -1//5)} => [5.01032e-08-1